In [1]:
# https://ai.google/tools/datasets/google-facial-expression/
import imageio
import os
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import random
import hashlib
from skimage.transform import resize
import math

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K

In [3]:
train_df = pd.read_csv('FEC_dataset/two_class_ready.csv')
train_df.head()

,Image1,Image2,Image3
0,22d27258ff7e78577c811c5b8187446f.jpg,5806875441d55aa9c1d8a8998d160a82.jpg,bb3f152775a40c53daafab83a43f5254.jpg
1,e5a2d4733125b8aedb18c2d8040868e6.jpg,8ed9229eb118373590a069b774aeeeec.jpg,9d5910b05710e09d2ab12e4d4b224938.jpg
2,e5a2d4733125b8aedb18c2d8040868e6.jpg,53e165101543063e4b4946eb411ebecd.jpg,bad09c8ea230ee5919fcf794ea7f4f4b.jpg
3,e5a2d4733125b8aedb18c2d8040868e6.jpg,0d3252b4ad950e4e875fd2ace92c9641.jpg,9f99820796b92f6566a4639028ad0a00.jpg
4,e5a2d4733125b8aedb18c2d8040868e6.jpg,59d6ef6f3971c8a828fb67b9961dba23.jpg,08d4159f45a1607a0aa00bd6dc75c24f.jpg


In [14]:
def TripletGen(data, root='FEC_dataset/formatted', batch_size=128, rescale=1./255, shape=(224,224)):
    # yields anchor, positive and negative examples in batches of @batch_size
    # output shape is (3*batch_size, 224, 224, 3)
    n = len(data)
    iterations = math.ceil(n/batch_size)
    print(f"Found {n} triplets in the dataset")
    # the main "yielding" loop
    for i in range(iterations):
        # use start and end as indices to work with
        start, end = i*batch_size, min((i+1)*batch_size, n)
        batch_size = (end - start)
        bs = 3*batch_size
        # yield X each time
        X = np.empty((bs,224, 224, 3))
        u, v = list(range(0,bs)), []
        for j in range(0,bs,3):
            # X[j] row is a triplet of size (224, 224, 3)
            for t in range(1,4):
                path = os.path.join(root, data[f'Image{t}'][start+j])
                temp = image.load_img( path, target_size = shape )
                X[j+t-1] = image.img_to_array(temp)*rescale
                v.append(j+t-1)
        yield X, tf.ones(X.shape, tf.int32)

# imgs = TripletGen(train_df)
# for i in range(5):
#     print(next(imgs).shape)

In [15]:
def batch_tf_triplet_loss(inputs, labels=None, alpha=0.2):
    sigma_dap = tf.math.reduce_sum(tf.norm(inputs[1::3]-inputs[0::3], axis=1))
    sigma_dan = tf.math.reduce_sum(tf.norm(inputs[2::3]-inputs[0::3], axis=1)+alpha)
    loss = sigma_dan - sigma_dap
    return tf.maximum(loss,0)

a, p, n = [1,2,3], [1,2, 4], [2,2,6]

inputs = tf.constant([a,p,n,a,p,n], tf.float64)
answer = batch_tf_triplet_loss(inputs)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    answer = sess.run(answer)
    print(answer)

4.724555320336759


In [16]:
model = tf.keras.models.Sequential([
    #first convolution
#     Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
#     MaxPool2D(2,2),
#     #second
#     Conv2D(32, (3,3), activation='relu'),
#     MaxPool2D(2,2),
#     #third
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPool2D(2,2),
#     #fourth
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPool2D(2,2),
#     #fifth
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPool2D(2,2),
    #hidden
    Flatten(input_shape=(224,224,3)),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'), #activity_regularizer=regularizers.l2(0.01)),
    #Last layer
    Dense(16)
])
from tensorflow.keras.optimizers import RMSprop
# model.compile(loss=triplet_loss, optimizer=RMSprop(lr=0.001), metrics=['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              154141696 
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 16)                16400     
Total params: 155,207,696
Trainable params: 155,207,696
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss=batch_tf_triplet_loss, optimizer=RMSprop(lr=0.001))

In [ ]:
train = TripletGen(train_df)

history = model.fit_generator(train, epochs=20, steps_per_epoch=30, verbose=1)
# with tf.Session() as sess:
#     print(sess.run(batch_tf_triplet_loss(model.predict(next(train)))))

Epoch 1/20
Found 136379 triplets in the dataset
